In [18]:
import numpy as np
import sqlite3
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

In [19]:
connection = sqlite3.connect(r"C:\Users\Igor\Documents\GitHub\Portfolio\website\static\data\database.sqlite")

players_df = pd.read_sql_query("SELECT * FROM Player", connection)
stats_df = pd.read_sql_query("SELECT * FROM Player_Attributes", connection)

In [20]:
players_df

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154
...,...,...,...,...,...,...,...
11055,11071,26357,Zoumana Camara,2488,1979-04-03 00:00:00,182.88,168
11056,11072,111182,Zsolt Laczko,164680,1986-12-18 00:00:00,182.88,176
11057,11073,36491,Zsolt Low,111191,1979-04-29 00:00:00,180.34,154
11058,11074,35506,Zurab Khizanishvili,47058,1981-10-06 00:00:00,185.42,172


In [21]:
stats_df

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183973,183974,102359,39902,2009-08-30 00:00:00,83.0,85.0,right,medium,low,84.0,...,88.0,83.0,22.0,31.0,30.0,9.0,20.0,84.0,20.0,20.0
183974,183975,102359,39902,2009-02-22 00:00:00,78.0,80.0,right,medium,low,74.0,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183975,183976,102359,39902,2008-08-30 00:00:00,77.0,80.0,right,medium,low,74.0,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183976,183977,102359,39902,2007-08-30 00:00:00,78.0,81.0,right,medium,low,74.0,...,88.0,53.0,28.0,32.0,30.0,9.0,20.0,73.0,20.0,20.0


In [22]:
def onehot_encode(df, column):
    dummies = pd.get_dummies(df[column], prefix=column)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [23]:
def preprocess_inputs(players, stats):
    players = players.drop(['id', 'player_name', 'player_fifa_api_id'], axis=1)
    stats = stats.drop(['id', 'player_fifa_api_id', 'date'], axis=1)

    players['birthday'] = pd.to_datetime(players['birthday'])
    players['birth_year'] = players['birthday'].apply(lambda x: x.year)
    players['birth_month'] = players['birthday'].apply(lambda x: x.month)
    players['birth_day'] = players['birthday'].apply(lambda x: x.day)
    players = players.drop('birthday', axis=1)

    categoricals = stats.groupby(by='player_api_id', as_index=False)[['player_api_id', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate']].head(1)

    for column in ['attacking_work_rate', 'defensive_work_rate']:
        categoricals[column] = categoricals[column].apply(lambda x: np.NaN if x not in ['low', 'medium', 'high'] else x)
        categoricals[column] = categoricals[column].fillna(categoricals[column].mode()[0])

    stats = stats.groupby(by='player_api_id').mean()
    stats = stats.merge(categoricals, on='player_api_id')
    
    for column in stats.loc[:, stats.isna().sum() > 0].columns:
        stats[column] = stats[column].fillna(stats[column].mean())

    df = players.merge(stats, on='player_api_id')
    df = df.drop('player_api_id', axis=1)

    df['preferred_foot'] = df['preferred_foot'].replace({'left': 0, 'right': 1})

    for column in ['attacking_work_rate', 'defensive_work_rate']:
        df = onehot_encode(df, column=column)
    
    y = df['overall_rating']
    x = df.drop('overall_rating', axis=1)

    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, shuffle=True, random_state=1)

    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = pd.DataFrame(scaler.transform(x_train), index=x_train.index, columns=x_train.columns)
    x_test = pd.DataFrame(scaler.transform(x_test), index=x_test.index, columns=x_test.columns)
    
    return x_train, x_test, y_train, y_test

In [24]:
x_train, x_test, y_train, y_test = preprocess_inputs(players_df, stats_df)

In [25]:
x_train

,height,weight,birth_year,birth_month,birth_day,potential,crossing,finishing,heading_accuracy,short_passing,...,gk_kicking,gk_positioning,gk_reflexes,preferred_foot,attacking_work_rate_high,attacking_work_rate_low,attacking_work_rate_medium,defensive_work_rate_high,defensive_work_rate_low,defensive_work_rate_medium
5429,0.149907,-0.033384,0.076129,0.306169,1.310542,0.035981,-2.226408,-1.761449,-2.491015,-2.570139,...,2.500292,2.649907,3.479833,0.564172,-0.520873,-0.235574,0.601974,-0.40871,-0.323411,0.55860
5874,0.944722,1.230136,0.259888,-0.858476,-0.392766,1.308621,0.061943,1.711121,1.384466,0.416901,...,-0.231833,-0.429216,-0.042825,-1.772509,-0.520873,-0.235574,0.601974,-0.40871,3.092040,-1.79019
755,0.149907,-0.033384,0.443648,-0.276154,-0.392766,-1.309262,-0.887226,0.917262,0.146312,-0.619679,...,-0.590616,-0.445620,-0.544244,0.564172,1.919853,-0.235574,-1.661202,-0.40871,-0.323411,0.55860
6074,0.944722,0.831130,0.259888,-0.276154,-0.165659,-1.265387,-0.887784,-1.025011,0.557556,-0.269012,...,-0.798593,-0.557207,-0.245866,-1.772509,-0.520873,-0.235574,0.601974,-0.40871,-0.323411,0.55860
2487,-1.042316,-0.764895,1.362444,1.761975,-0.052105,0.320386,0.377154,0.674039,-1.586912,0.409526,...,-0.516155,-0.305421,-0.186728,0.564172,-0.520873,-0.235574,0.601974,-0.40871,-0.323411,0.55860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7813,0.149907,-0.232887,-0.291389,0.306169,0.175003,-2.092746,-2.140441,-1.863503,0.506799,-0.549277,...,-0.798593,-0.305421,-0.659832,0.564172,-0.520873,-0.235574,0.601974,-0.40871,-0.323411,0.55860
10955,-0.247501,-1.230403,1.362444,0.015008,0.515665,0.369634,0.245573,0.256369,-0.852754,0.672933,...,-0.483876,-0.080613,-0.507763,-1.772509,-0.520873,-0.235574,0.601974,-0.40871,-0.323411,0.55860
905,-0.644908,-0.232887,1.729962,0.015008,0.175003,0.010126,1.175416,-1.036043,0.316462,-0.180507,...,-0.798593,-0.242475,-0.068452,0.564172,-0.520873,-0.235574,0.601974,-0.40871,-0.323411,0.55860
5192,-1.042316,-1.695910,-1.577704,1.470814,-0.165659,0.772848,1.198443,1.294633,0.736790,0.455622,...,1.545649,0.127334,0.035040,-1.772509,-0.520873,-0.235574,0.601974,-0.40871,-0.323411,0.55860


In [26]:
y_train

5429     68.450000
5874     75.433333
755      59.954545
6074     61.600000
2487     66.000000
           ...    
7813     59.333333
10955    65.285714
905      63.000000
5192     72.500000
235      74.200000
Name: overall_rating, Length: 7741, dtype: float64

In [27]:
model = RandomForestRegressor()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
rmse = np.sqrt(np.mean((y_test - y_pred)**2))
r2 = model.score(x_test, y_test)

print("Root Mean Square Error: {:.3f}".format(rmse))
print("R2 Score: {:.3f}".format(r2))

Root Mean Square Error: 1.529
R2 Score: 0.940


In [28]:
import pickle

pickle.dump(model, open('model.pkl', 'wb'))

In [29]:
def onehot_encode(df, column):
    dummies = pd.get_dummies(df[column], prefix=column)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

def preprocess_input(player):
    
    connection = sqlite3.connect(r"C:\Users\Igor\Documents\GitHub\Portfolio\website\static\data\database.sqlite")
    players = pd.read_sql_query("SELECT * FROM Player", connection)
    stats = pd.read_sql_query("SELECT * FROM Player_Attributes", connection)
    
    players = players.drop(['id', 'player_fifa_api_id'], axis=1)
    stats = stats.drop(['id', 'player_fifa_api_id', 'date'], axis=1)

    players['birthday'] = pd.to_datetime(players['birthday'])
    players['birth_year'] = players['birthday'].apply(lambda x: x.year)
    players['birth_month'] = players['birthday'].apply(lambda x: x.month)
    players['birth_day'] = players['birthday'].apply(lambda x: x.day)
    players = players.drop('birthday', axis=1)

    categoricals = stats.groupby(by='player_api_id', as_index=False)[['player_api_id', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate']].head(1)

    for column in ['attacking_work_rate', 'defensive_work_rate']:
        categoricals[column] = categoricals[column].apply(lambda x: np.NaN if x not in ['low', 'medium', 'high'] else x)
        categoricals[column] = categoricals[column].fillna(categoricals[column].mode()[0])

    stats = stats.groupby(by='player_api_id').mean()
    stats = stats.merge(categoricals, on='player_api_id')
    
    for column in stats.loc[:, stats.isna().sum() > 0].columns:
        stats[column] = stats[column].fillna(stats[column].mean())

    df = players.merge(stats, on='player_api_id')
    df = df.drop('player_api_id', axis=1)

    index_wanted = df.loc[df['player_name'] == player].index
    df = df.drop('player_name', axis=1)

    df['preferred_foot'] = df['preferred_foot'].replace({'left': 0, 'right': 1})

    for column in ['attacking_work_rate', 'defensive_work_rate']:
        df = onehot_encode(df, column=column)

    df = df.drop('overall_rating', axis=1)

    scaler = StandardScaler()
    scaler.fit(df)
    df = pd.DataFrame(scaler.transform(df), index=df.index, columns=df.columns)
    df_final = df.iloc[index_wanted].values.reshape(1, -1)
  
    x = pd.DataFrame(df_final, columns=df.columns)

    return x


In [30]:
x = preprocess_input('Cristiano Ronaldo')
prediction = round( float(model.predict(x)), 2 )
prediction

90.04